In [1]:
from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('/content/drive/My Drive/')

In [0]:
os.chdir('ir_project/New folder/')

In [4]:
!pip install transformers

     |████████████████████████████████| 645kB 2.7MB/s 
     |████████████████████████████████| 1.1MB 6.8MB/s 
     |████████████████████████████████| 890kB 14.6MB/s 
     |████████████████████████████████| 3.8MB 17.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=82c94ba8e7b2960c6c643048548a3fc9011da310f110f9ec7f49142492e69462
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [5]:

import transformers
import pandas as pd
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

from torch.utils import data
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F



class TrainLoader(data.Dataset):

  def __init__(self,sentences,aspect_labels,sentiments,attention_mask):

    self.sentences=sentences
    self.aspect_labels=aspect_labels
    self.sentiments=sentiments
    self.attention_mask=attention_mask
    
    self.length=len(self.sentences)


  def __getitem__(self,idx):
      
      sentence = self.sentences[idx]
      aspect_label = self.aspect_labels[idx]
      sentiment = self.sentiments[idx]
      attention_mask = self.attention_mask[idx]
      
      return sentence,aspect_label,sentiment,attention_mask

  def __len__(self):
      return self.length

            
class Model(nn.Module):
    
    def __init__(self):
        super(Model, self).__init__()
        self.model_class, self.tokenizer_class, self.pretrained_weights = (transformers.DistilBertModel, transformers.DistilBertTokenizer, 'distilbert-base-uncased')
        self.tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
        self.model = model_class.from_pretrained(pretrained_weights)
        
        self.drop_out=nn.Dropout(0.1)
        self.conv1=nn.Conv1d(768*2,768,1)
        
        self.output=nn.Linear(768,82)
        
        self.layer1=nn.Linear(82,82)
        self.sentiment_output = nn.Linear(82,82*4)
        
    
    def forward(self,x,aspect_label,attention_masks):
        
        aspect_label.requires_grad=False
        x = self.model(x,attention_masks)
        
        x = torch.cat((x[0][:,-1,:],x[0][:,-2,:]),dim=-1)
        
        x = x.unsqueeze(2)
        
        x=self.drop_out(x)
        
        x=self.conv1(x)
        x=F.relu(x)
        #print(x.shape)
        x=x.squeeze(2)
        x=self.output(x)
        x=F.relu(x)
        x = aspect_label.view(aspect_label.shape[0]*82)*x.view(aspect_label.shape[0]*82)
        
        x= x.view(aspect_label.shape[0],82)
        
        x= self.layer1(x)
        x=F.relu(x)
        
        x = self.sentiment_output(x)
        
        #print(x.shape)
        
        x=x.view(aspect_label.shape[0],82,4)
        
        #x=F.softmax(x,dim=2)
        
        return x
    
model_class, tokenizer_class, pretrained_weights = (transformers.DistilBertModel, transformers.DistilBertTokenizer, 'distilbert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

train_data = np.load('train_data.npy',allow_pickle=True)

sentiment_mapping={}
sentiment_mapping['positive']=1
sentiment_mapping['negative']=2
sentiment_mapping['neutral']=3

all_labels=[]
for row in train_data:
    if row[-1]=='':
        continue
    else:
        all_labels.extend(row[-1]['category'])

all_labels=list(set(all_labels))

labels_mapping={}
idx=0

for label in all_labels:
    labels_mapping[label]=idx
    idx+=1
    
aspect_labels=[]
sentiment=[]
sentences=[]
for row in train_data:
    
    if row[-1]=='':
        continue
    else:
        labels=row[-1]['category']
        sentiments=row[-1]['polarity']
        a=[]
        s=[]
        for i in labels:
            a.append(labels_mapping[i])
        for j in sentiments:
            s.append(sentiment_mapping[j])
        label = a
    sentence=row[2]
    sentences.append(sentence)
    aspect_labels.append(label)
    sentiment.append(s)

print(len(sentences),len(aspect_labels),len(sentiment))

classes = len(all_labels)+1

new_aspect_labels=[]
new_sentiments =[]
#https://discuss.pytorch.org/t/what-kind-of-loss-is-better-to-use-in-multilabel-classification/32203/4

for label,sen in zip(aspect_labels,sentiment):
    a=torch.LongTensor(label)
    a=a.unsqueeze(0)
    new = np.zeros((classes,1))
    
    
    for index,idc in zip(label,sen):
        new[index][0]=idc
    new_sentiments.append(new)
    target = torch.zeros(a.size(0),82).scatter_(1, a, 1.)
    new_aspect_labels.append(target.numpy()[0])

new_aspect_labels=torch.FloatTensor(np.array(new_aspect_labels))

new_sentiments=torch.FloatTensor(np.array(new_sentiments))

tokenized_sentences = []

for s in sentences :
    if isinstance(s,float):
        print("hi there!")
        continue
    a=tokenizer.encode(s,add_sepcial_tokens=True)
    tokenized_sentences.append(a)

max_len = 0
for i in tokenized_sentences:
    if len(i) > max_len:
        max_len = len(i)


padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized_sentences])
input_ids = torch.LongTensor(np.array(padded))
attention_mask = np.where(padded != 0, 1, 0)

train_target=TrainLoader(input_ids,new_aspect_labels,new_sentiments,attention_mask)

train_loader=torch.utils.data.DataLoader(train_target,batch_size=16,shuffle=True)

criterion = nn.CrossEntropyLoss()
#train
net=Model()
net=net.cuda()

optimizer = optim.Adadelta(net.parameters(), lr=1.0, rho=0.95, eps=1e-06, weight_decay=0)

for epoch in range(20):
    total_loss=0
    for sentence,aspect_label,sentiment,mask in train_loader:
        
        pred=net(sentence.cuda(),aspect_label.cuda(),mask.cuda())
        
        loss = criterion(pred[0],sentiment[0].squeeze(1).long().cuda())
        
        optimizer.zero_grad()
        
        for i in range(1,len(pred)):
            loss += criterion(pred[i],sentiment[i].squeeze(1).long().cuda())
        
        loss.backward()
        
        optimizer.step()
        total_loss+=loss.item()
        print(loss.item())
    print("epoch : ",epoch," loss : ",loss.item())

        


1399 1399 1399
22.273544311523438
21.964826583862305
21.620933532714844
21.22203254699707
20.53208351135254
18.989126205444336
12.991625785827637
10.127161979675293
9.14474105834961
6.165490627288818
10.179289817810059
2.1570541858673096
5.739212989807129
8.297473907470703
7.666986465454102
6.065352439880371
6.609152793884277
3.8606364727020264
5.24063777923584
4.469970226287842
3.002723217010498
1.9643733501434326
3.9918997287750244
2.817574977874756
3.4869003295898438
2.694803476333618
2.8131368160247803
4.284448623657227
5.273437023162842
2.2914481163024902
3.7653698921203613
4.537769794464111
2.9353208541870117
1.791399598121643
2.750518560409546
3.0999255180358887
1.6230287551879883
0.9465028643608093
2.2024452686309814
2.4730417728424072
1.7740669250488281
1.8086868524551392
1.4608608484268188
1.4973500967025757
1.250342845916748
1.6856001615524292
1.476730227470398
1.0481544733047485
0.7972676157951355
1.6307258605957031
1.284990906715393
1.1798776388168335
1.5724272727966309
1

In [6]:
predictions=[]
labels =[]
for sentence,aspect_label,sentiment,mask in train_loader:
        
        pred=net(sentence.cuda(),aspect_label.cuda(),mask.cuda())
        preds = torch.argmax(pred,dim=2)
        
        p=preds.cpu().detach().numpy().flatten()
        predictions.extend(p)

        labels.extend(sentiment.squeeze(2).numpy().flatten())

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

labels = np.array(labels)
predictions = np.array(predictions)

c=classification_report(labels,predictions)

print(c.split('\n'))


['              precision    recall  f1-score   support', '', '         0.0       0.99      1.00      1.00    112746', '         1.0       0.82      0.61      0.70      1102', '         2.0       0.59      0.36      0.45       764', '         3.0       0.56      0.08      0.15       106', '', '    accuracy                           0.99    114718', '   macro avg       0.74      0.52      0.57    114718', 'weighted avg       0.99      0.99      0.99    114718', '']


In [7]:
c.split("\n")

['              precision    recall  f1-score   support',
 '',
 '         0.0       0.99      1.00      1.00    112746',
 '         1.0       0.82      0.61      0.70      1102',
 '         2.0       0.59      0.36      0.45       764',
 '         3.0       0.56      0.08      0.15       106',
 '',
 '    accuracy                           0.99    114718',
 '   macro avg       0.74      0.52      0.57    114718',
 'weighted avg       0.99      0.99      0.99    114718',
 '']

In [0]:
torch.save(net.state_dict(), "sentiment_analysis.pt")

In [9]:

import transformers
import pandas as pd
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

from torch.utils import data
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F


test_data = np.load('test_data.npy',allow_pickle=True)

aspect_labels=[]
sentiment=[]
sentences=[]
for row in test_data:
    
    if row[-1]=='':
        continue
    else:
        labels=row[-1]['category']
        sentiments=row[-1]['polarity']
        a=[]
        s=[]
        for i in labels:
          if i not in labels_mapping.keys():
            continue
          a.append(labels_mapping[i])
        for j in sentiments:
            s.append(sentiment_mapping[j])
        label = a
    if a!=[]:
      sentence=row[2]
      sentences.append(sentence)
      aspect_labels.append(label)
      sentiment.append(s)

print(len(sentences),len(aspect_labels),len(sentiment))

classes = len(all_labels)+1

new_aspect_labels=[]
new_sentiments =[]
#https://discuss.pytorch.org/t/what-kind-of-loss-is-better-to-use-in-multilabel-classification/32203/4

for label,sen in zip(aspect_labels,sentiment):
    a=torch.LongTensor(label)
    a=a.unsqueeze(0)
    new = np.zeros((classes,1))
    
    
    for index,idc in zip(label,sen):
        new[index][0]=idc
    new_sentiments.append(new)
    target = torch.zeros(a.size(0),82).scatter_(1, a, 1.)
    new_aspect_labels.append(target.numpy()[0])

new_aspect_labels=torch.FloatTensor(np.array(new_aspect_labels))

new_sentiments=torch.FloatTensor(np.array(new_sentiments))

tokenized_sentences = []

for s in sentences :
    if isinstance(s,float):
        print("hi there!")
        continue
    a=tokenizer.encode(s,add_sepcial_tokens=True)
    tokenized_sentences.append(a)

max_len = 0
for i in tokenized_sentences:
    if len(i) > max_len:
        max_len = len(i)


padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized_sentences])
input_ids = torch.LongTensor(np.array(padded))
attention_mask = np.where(padded != 0, 1, 0)

test_target=TrainLoader(input_ids,new_aspect_labels,new_sentiments,attention_mask)

test_loader=torch.utils.data.DataLoader(test_target,batch_size=16,shuffle=True)

#train
predictions=[]
labels =[]
for sentence,aspect_label,sentiment,mask in test_loader:
        
        pred=net(sentence.cuda(),aspect_label.cuda(),mask.cuda())
        preds = torch.argmax(pred,dim=2)
        
        p=preds.cpu().detach().numpy().flatten()
        predictions.extend(p)

        labels.extend(sentiment.squeeze(2).numpy().flatten())

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

labels = np.array(labels)
predictions = np.array(predictions)

c_test=classification_report(labels,predictions)

print(c.split('\n'))


643 643 643
['              precision    recall  f1-score   support', '', '         0.0       0.99      1.00      1.00    112746', '         1.0       0.82      0.61      0.70      1102', '         2.0       0.59      0.36      0.45       764', '         3.0       0.56      0.08      0.15       106', '', '    accuracy                           0.99    114718', '   macro avg       0.74      0.52      0.57    114718', 'weighted avg       0.99      0.99      0.99    114718', '']


In [10]:
c_test.split('\n')

['              precision    recall  f1-score   support',
 '',
 '         0.0       0.99      1.00      1.00     51784',
 '         1.0       0.65      0.55      0.60       538',
 '         2.0       0.47      0.30      0.36       325',
 '         3.0       0.00      0.00      0.00        79',
 '',
 '    accuracy                           0.99     52726',
 '   macro avg       0.53      0.46      0.49     52726',
 'weighted avg       0.99      0.99      0.99     52726',
 '']